# Assignment 6
###Mark Swanson
###COP 58959 - Summer 2015
###Dr. Shankar
###2015-07-28

Retrieve SMART on FHIR datasets for patients' vital signs utilizing the smarthealthit.org API. After collecting the data, plot histogram data of vital signs for all patients utiliznig the FHIR namespace.

Import the necessary modules

In [1]:
from xml.etree.ElementTree import parse
import subprocess
from pandas import DataFrame
import matplotlib.pyplot as plt
import os
import errno

Initialize global variables and generate necessary temporary filesystem

In [2]:
mean_bp = []
mean_diastolic = []
mean_systolic = []
count = 0

try:
    os.makedirs('tmp')
except OSError:
    if not os.path.isdir('tmp'):
        raise

Retrieve a list of patients from SMART and generate an Observation file for each patient containing vital sign data

In [3]:
patient_file = open('tmp/patient_file', 'w')
subprocess.call(['curl', '-s', '-k', '-H', '"Accept: application/json+fhir"', 'https://fhir-open-api.smarthealthit.org/Patient'], stdout=patient_file)
patient_file.close()

patient_parse = parse('tmp/patient_file')

patient_list = []
for identifier in patient_parse.findall('.//{http://hl7.org/fhir}identifier'):
  value = identifier.find('{http://hl7.org/fhir}value')
  #print value.get('value')
  patient_list.append(value.get('value'))
#print patient_list

#patient_list = ['1288992']
for patient in patient_list:
  patient_file = open('tmp/' + patient, 'w')
  subprocess.call(['curl', '-s', '-k', '-H', '"Accept: application/json+fhir"', 'https://fhir-open-api.smarthealthit.org/Observation/_search?subject%3APatient=' + patient + '&name=8302-2%2C3141-9%2C8867-4%2C9279-1%2C8310-5%2C39156-5%2C2710-2%2C8287-5%2C55284-4%2C8480-6%2C8462-4'], stdout=patient_file)
  patient_file.close()
  #print patient

Calculate mean systolic, diastolic, and mean blood pressures for each patient, trimming the data as necessary as only the previous 50 values are available, and store in global arrays for plotting

In [4]:
for patient in patient_list:
  doc = parse('tmp/' + patient)
  
  systolic_bps = []
  diastolic_bps = []

  for observation in doc.findall('.//{http://hl7.org/fhir}Observation'):
    text_ele = observation.find('{http://hl7.org/fhir}text')
    div = text_ele.find('{http://www.w3.org/1999/xhtml}div')
    if div.text.find('Systolic') != -1:
      value_quant = observation.find('{http://hl7.org/fhir}valueQuantity')
      value = value_quant.find('{http://hl7.org/fhir}value')
      systolic_bps.append(value.get('value'))
    if div.text.find('Diastolic') != -1:
      value_quant = observation.find('{http://hl7.org/fhir}valueQuantity')
      value = value_quant.find('{http://hl7.org/fhir}value')
      diastolic_bps.append(value.get('value'))

  if (not systolic_bps) & (not diastolic_bps):
    #print "No Systolic/Diastolic BP"
    continue
  
  while len(systolic_bps) > len(diastolic_bps):
    systolic_bps.pop()   
  while len(diastolic_bps) > len(systolic_bps):
    diastolic_bps.pop()
    
  enc_dict = {}
  enc_dict['systolic_bp'] = systolic_bps
  enc_dict['diastolic_bp'] = diastolic_bps
  encounters = DataFrame(enc_dict, columns=['diastolic_bp', 'systolic_bp'])
  encounters = encounters.convert_objects(convert_dates='coerce', convert_numeric=True)
  #print encounters
  
  enc_period = encounters
  enc_period['mean_bp'] = enc_period['diastolic_bp'] + ((enc_period['systolic_bp']-enc_period['diastolic_bp'])/3)
  
  mbp = enc_period['mean_bp'].mean()
  msys = enc_period['systolic_bp'].mean()
  mdia = enc_period['diastolic_bp'].mean()
  mean_bp.append(mbp)
  mean_systolic.append(msys)
  mean_diastolic.append(mdia)
  count = count + 1
  #print mbp
  #print msys
  #print mdia

Plot mean systolic, diastolic, and mean blood pressure histograms for the patient list

In [5]:
#print mean_bp
#print mean_systolic
#print mean_diastolic
#print count

plt.figure(1)
plt.hist(mean_bp, stacked=True)
plt.title("Gaussian Histogram")
plt.xlabel("Mean Blood Pressure")
plt.ylabel("Frequency")

plt.figure(2)
plt.hist(mean_systolic, stacked=True)
plt.title("Gaussian Histogram")
plt.xlabel("Mean Systolic Pressure")
plt.ylabel("Frequency")

plt.figure(3)
plt.hist(mean_diastolic, stacked=True)
plt.title("Gaussian Histogram")
plt.xlabel("Mean Diastolic Pressure")
plt.ylabel("Frequency")

plt.show()
plt.close()

Clean up working directory

In [6]:
for root, dirs, files in os.walk('tmp', topdown=False):
    for name in files:
        os.remove(os.path.join(root, name))
    for name in dirs:
        os.rmdir(os.path.join(root, name))
    os.rmdir(root)